In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Load API key from .env file

In [2]:
KEY = os.environ.get("GEMINI_API_KEY")

Gemini

In [3]:
from google import genai
from google.genai import types

In [4]:
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client(api_key=KEY)  # here you can also pass the api_key directly using os.environ['GEMINI_API_KEY']

default_model = "gemini-2.5-flash"

In [5]:
response = client.models.generate_content(
    model=default_model,
    contents="Explain cat photos",
    config=types.GenerateContentConfig(
        max_output_tokens=50,
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
)
print(response.text)

Cat photos are an internet phenomenon and a cultural touchstone. They are **pictures or videos of domestic felines**, often engaged in:

* **Typical cat behaviors:** Sleeping in strange positions, grooming, eating, looking out a window.
*


RAG

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # get embeddings from Gemini
from langchain_community.vectorstores import FAISS  # "db" to store and retrieve embeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter  # split long documents
from pdfminer.high_level import extract_text  # extract text from pdfs
from langchain_huggingface import HuggingFaceEmbeddings

In [10]:
DOC_PATH = "./scraped_pdfs/32018R0213.pdf"

In [11]:
import logging
# Silence pdfminer logs below ERROR
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# Extract text
text = extract_text(DOC_PATH, page_numbers=[0])
# text = extract_text(DOC_PATH)
print(text)

L  41/6 

SV     

Europeiska unionens  officiella  tidning 

14.2.2018 

KOMMISSIONENS FÖRORDNING  (EU)  2018/213 

av  den 12 februari  2018 

om  användning  av  bisfenol  A  i  lack  och  ytskikt  avsedda  att  komma  i  kontakt  med  livsmedel  och 
om  ändring  av  förordning  (EU)  nr  10/2011  vad  gäller  användningen  av  det  ämnet  i  plastmaterial 
avsedda att komma i  kontakt  med livsmedel 

(Text  av betydelse för EES) 

EUROPEISKA  KOMMISSIONEN  HAR  ANTAGIT  DENNA FÖRORDNING 

med  beaktande av fördraget om Europeiska unionens  funktionssätt, 

med  beaktande  av  Europaparlamentets  och  rådets  förordning  (EG)  nr  1935/2004  av  den  27  oktober  2004  om  material 
och  produkter  avsedda  att  komma  i  kontakt  med  livsmedel  och  om  upphävande  av  direktiven  80/590/EEG  och 
89/109/EEG  (1),  särskilt  artikel  5.1  d, e, h,  i och  j,  och 

av följande  skäl: 

(1)  

(2)  

(3)  

Ämnet  2,2-bis(4-hydroxifenyl)propan 
(CAS-nr  0000080-05-7),  vanligen  

In [12]:
# Wrap as a Document
doc = Document(page_content=text, metadata={"source": "32018R0213.pdf", "page": 1})

In [13]:
# 2. Split the documents into small chunks

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs = splitter.split_documents([doc])

for i, d in enumerate(docs, 1):
   print(f"Chunk {i}: {d.page_content[:60]}... | Metadata: {d.metadata}")

Chunk 1: L  41/6 

SV     

Europeiska unionens  officiella  tidning ... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 2: om  användning  av  bisfenol  A  i  lack  och  ytskikt  avse... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 3: om  ändring  av  förordning  (EU)  nr  10/2011  vad  gäller ... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 4: (Text  av betydelse för EES) 

EUROPEISKA  KOMMISSIONEN  HAR... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 5: med  beaktande  av  Europaparlamentets  och  rådets  förordn... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 6: och  produkter  avsedda  att  komma  i  kontakt  med  livsme... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 7: av följande  skäl: 

(1)  

(2)  

(3)... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 8: Ämnet  2,2-bis(4-hydroxifenyl)propan 
(CAS-nr  0000080-05-7)... | Metadata: {'source': '32018R0213.pdf', 'page': 1}
Chunk 9: tillverkning 

In [14]:
# Local embedding model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [15]:
# Database
db = FAISS.from_documents(docs, embeddings)

In [22]:
# 5. Given a query, retrieve the most relevant chunk(s) and appropriately prompt your LLM

query = "Bisfenol A"
results = db.similarity_search(query, k=10)

for r in results:
    print(r.page_content)

Ämnet  2,2-bis(4-hydroxifenyl)propan 
(CAS-nr  0000080-05-7),  vanligen  kallat  bisfenol  A,  används  vid
(3) Opinion of the Scientific Committee on Food on Bisphenol A (SCF/CS/PM/3936 Final). 
(4) The EFSA Journal, nr 428, s. 1, 2006. 
(5) The EFSA Journal, nr 759, s. 1, 2008.
migration  på  0,6  mg  bisfenol  A  per  kg  livsmedel  (mg/kg)  baserat  på  en  föregående  utvärdering  av  vetenskapliga
Efter  offentliggörandet  av  myndighetens  vetenskapliga  yttrande  om  bisfenol  A  2011  konstaterade  myndigheten
2011  (7).  I  enlighet  med  försiktighetsprincipen  gäller  ett  förbud  mot  användning  av  bisfenol  A vid  tillverkning  av 
nappflaskor av polykarbonat  för  spädbarn.
de  vetenskapliga  uppgifterna  och  uppdaterade  sitt  yttrande  om  bisfenol  A  2006  (4),  2008  (5),  2010  (6)  och
betydelsen 
exponeringsvägar  också  bör 
undersökas.  Myndigheten  beslutade  att  genomföra  en  fullständig  omprövning  av  bisfenol  A  utifrån  de  senaste
om  användning  

GUI

In [ ]:
import gradio as gr